# Exercise 2 - Unlinkable proofs using BBS+

In this exercise, we will explore the BBS cryptographic scheme, and see how it differs from a classic RSA scheme as explored in the previous exercise.

For this, we will use the [docknetwork crypto library](https://github.com/docknetwork/crypto). 
The authors of this crypto library are from Dock.io, which is one of the leading startups in the Crypto E-ID space.  
They open-source many fundamental libraries including the one we're using it today.

Dock.io have listed 16 employess published on their website and are originated in Switzerland, but are now USA based.
They got a 20 million USD fund in a series A with the Web3 foundation as lead investor.

## Sections

1. Credential - Setting up a corresponding JSON Schema
2. Issuer - Setting up and creating a BBS+ signature
3. Holder - Creating a proof
4. Verifier - Verifying the proof is valid
5. Discussion - Unlinkability achieved?
6. Coding exercise - Simulate an issuer-holder-verifier setup

---
## 1. Credential - Setting up a corresponding JSON Schema

The credential below has a corresponding JSON schema.
JSON Schema is the standard for defining the structure/ schema of a JSON object.
It's typically used to define a contract for APIs and data communication (that use JSON).  
One known standard that builds on JSON schema is the Open API Specification (OAS - previously known as swagger). 
OAS -in its latest versions- uses a superset of the latest published [JSON schema](https://json-schema.org/specification) (2020-12).

Here the JSON schema is used by the BBS+ library to correctly encode the different fields in the credential.
This will allow us in Exercise 3 to create zero knowledge proofs on this credential.

In [ ]:
// We start by declaring our most important asset which is the E-ID Verifiable Credential we will be working with.

const E_ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: new Date("1993-08-01T00:00:00").getTime(),
    profession: "IT Manager"
}

// For this advanced library, we will also need to specify the schema of Credential we're working with.
// Specifying the schema will be useful later to get the correct encoding of each field.
const E_ID_SCHEMA = {
    type: 'object',
    properties: {
        name: { type: 'string' },
        profession: { type: 'string' },
        timeOfBirth: { type: 'integer', minimum: 0},
    }
}

### Questions

1. What is the format for `timeOfBirth`? And what is the limitations using that format? What else could the format be?
2. What other fields could be in the credential?


---
## 2. Issuer - Setting up and creating a BBS+ signature


In [ ]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

import { BBSKeypair, BBSSignatureParams, BBS_SIGNATURE_PARAMS_LABEL_BYTES,
        CredentialSchema, BBSCredentialBuilder, BBSCredential, SUBJECT_STR } from '@docknetwork/crypto-wasm-ts'

const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

// Generating a keypair once signature parameters are created.
const params = BBSSignatureParams.generate(100, BBS_SIGNATURE_PARAMS_LABEL_BYTES);
const keypair = BBSKeypair.generate(params, stringToBytes('seed1'));

const secretKey = keypair.secretKey;
const publicKey = keypair.publicKey;

// Issuing a credential based on the E-ID data based on the specified schema

// sets a default structure for a credential with the proof 
let credential_schema = CredentialSchema.essential();
// Puts our data schema inside the credential schema
credential_schema.properties[SUBJECT_STR] = E_ID_SCHEMA

// Populate the credential with data
const builder = new BBSCredentialBuilder();
builder.schema = new CredentialSchema(credential_schema);
builder.subject = E_ID_DATA

// sign the credential
const verifiableCredential = builder.sign(secretKey);

// Now, the credential is sent to the holder
const credentialToSendToHolder = verifiableCredential.toJSON()
console.log("Credential sent to the holder:", credentialToSendToHolder)

### Questions

1. What information do you find in the credential which is sent to the holder? What keywords / algorithms do you recognize?
2. How big in bytes is the proof sent to the holder?

---
## 3. Holder - Creating a proof


In [ ]:
import { PresentationBuilder } from '@docknetwork/crypto-wasm-ts'

// The holder loads the credential back from JSON
const receivedCredential = BBSCredential.fromJSON(credentialToSendToHolder)

// They verify that the credential is valid:
console.log("It is a valid credential:", verifiableCredential.verify(publicKey).verified)

// The holder wants to prove their name and profession, but keep their birthdate secret
const presentationBuilder = new PresentationBuilder();
presentationBuilder.addCredential(receivedCredential, publicKey)
presentationBuilder.markAttributesRevealed(0, new Set<string>(['credentialSubject.name', 'credentialSubject.profession']));

const presentation = presentationBuilder.finalize();
const bbsProofForVerifier = presentation.toJSON()
console.log("The proof which is sent to the verifier:", bbsProofForVerifier);

### Holder -> Verifier

The following data is sent to the verifier
1. selected data to be revealed
2. Schema of the credential
3. Proof of validity

All this information and more is stored in the "presentation object". Note that this object is a completely serializable JSON object.


### Questions

1. Explore the bbsProofVerifier to see what information is sent to the verifier
2. Is there information which could be used to de-anonymize the holder?
3. How long is the proof now? Why is it bigger than the proof sent by the issuer?

---
## 4. Verifier - Verifying the proof is valid


In [ ]:
// Verification of the data the verifier received is as easy as two lines of code!
import { Presentation } from '@docknetwork/crypto-wasm-ts'

const recreatedPres = Presentation.fromJSON(bbsProofForVerifier)
console.log("The proof is valid:", recreatedPres.verify([publicKey]).verified);
console.log("The shared data is:", recreatedPres.spec.credentials[0].revealedAttributes);

### Questions

1. Can you create a proof which cannot be validated? What did you change?
2. Reveal another element of the credential - what did you need to change?
3. How does the verifier know the credentials to reveal?

---
## 5. Discussion - Unlinkability achieved?

We can know that by checking for two conditions

1. Is there a signature or a cryptopgraphic data that is shared and could used to identify the credential holder?
2. Is there a part of data hashed or not that was included in the presented data that could be used to identify the credential holder?

In [ ]:
// Run this a couple of times to see how it changes
console.log("One finalized proof:", presentationBuilder.finalize()["proof"].hex)

In [ ]:
// To answer the first point, we look at the presented proof 
const proof1 = presentationBuilder.finalize()["proof"].hex
const proof2 = presentationBuilder.finalize()["proof"].hex

// By generating a proof multiple times, and comparing the proof in each of them,
// we see that they're always different making it difficult (Impossible?!) to correlate the different proofs.
console.log("Two created proofs are the same:", proof1 === proof2);

// This is a weak proof! In the paper, there's a real mathematical proof to 
// ensure that different generated proofs are unlinkable 

In [ ]:
// To answer the second point, we look at the data shared with the verifier.
// As opposed to the previous exercise where we had to share all the data all the time
// even though possibly in a hashed format, in this case, we only share the selected data.

console.log("Revealed attributes:", presentation.spec.credentials[1]["revealedAttributes"]);

// In this scenario, we see that only the data needed to be revealed is revealed resulting in unlinkability.

---

## 6. Coding exercise - Simulate an issuer-holder-verifier setup

Divide into two groups of three teams (issuers (min:1), holders(min:2), verifiers(min:1))
and use a chat tool to communicate. 